In [2]:
!pip install transformers datasets accelerate ray[tune] --quiet

DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/igraph-0.11.8-py3.12-linux-x86_64.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/texttable-1.7.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/looseversion-1.3.0-py3.12.egg is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to use pip for package installation. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /usr/local/lib/python3.12/dist-packages/opt_einsum-3.4.0-py3.12.egg is deprecated. pip 25.1 will enf

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, DataCollatorForLanguageModeling
from datasets import load_dataset
import ray
from ray import tune, train
from ray.tune.schedulers import ASHAScheduler

device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

def get_model_and_tokenizer():
    tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
    tokenizer.pad_token = tokenizer.eos_token
    model = AutoModelForCausalLM.from_pretrained("distilgpt2")
    model.resize_token_embeddings(len(tokenizer))
    return model.to(device), tokenizer

def load_tokenized_dataset(tokenizer, block_size=64):
    raw_dataset = load_dataset("wikitext", "wikitext-2-raw-v1", split="train[:1%]")

    def tokenize(example):
        tokens = tokenizer(example["text"], padding="max_length", truncation=True, max_length=block_size)
        tokens["labels"] = tokens["input_ids"].copy() 
        return tokens

    tokenized = raw_dataset.map(tokenize, batched=True, remove_columns=["text"])
    tokenized.set_format("torch")
    return tokenized

def train_with_tune(config):
    print(f"🔧 Starting trial with config: {config}")
    try:
        model, tokenizer = get_model_and_tokenizer()
        dataset = load_tokenized_dataset(tokenizer)

        train_size = int(0.8 * len(dataset))
        train_dataset = dataset.select(range(train_size))
        eval_dataset = dataset.select(range(train_size, len(dataset)))

        print(f"📊 Train size: {len(train_dataset)}, Eval size: {len(eval_dataset)}")

        training_args = TrainingArguments(
            output_dir="./output",
            per_device_train_batch_size=config["batch_size"],
            learning_rate=config["lr"],
            num_train_epochs=config["epochs"],
            logging_steps=5,
            save_strategy="no",
            report_to="none",
            fp16=torch.cuda.is_available(),
        )

        # Use a data collator that supports masked LM tasks
        data_collator = DataCollatorForLanguageModeling(
            tokenizer=tokenizer, mlm=False  # GPT-style LM = causal, not masked
        )

        trainer = Trainer(
            model=model,
            args=training_args,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            data_collator=data_collator,
        )

        trainer.train()

        result = trainer.evaluate()
        print("📉 Eval results:", result)

        eval_loss = result.get("eval_loss", None)
        if eval_loss is None:
            print("⚠️ Warning: eval_loss missing. Reporting dummy loss = 9999.")
            eval_loss = 9999.0

        #tune.report(loss=eval_loss)
        train.report({"loss": eval_loss})

    except Exception as e:
        print(f"❌ Trial crashed: {e}")
        train.report({"loss": 9999.0})

# Define search space
search_space = {
    "batch_size": tune.choice([1, 2]),
    "lr": tune.loguniform(1e-5, 1e-4),
    "epochs": tune.choice([1, 2]),
}

scheduler = ASHAScheduler()

ray.shutdown()
ray.init(ignore_reinit_error=True, num_cpus=2)

# Run tuning
analysis = tune.run(
    train_with_tune,
    config=search_space,
    num_samples=2,
    scheduler=scheduler,
    metric="loss",
    mode="min",
    resources_per_trial={"cpu": 1, "gpu": 0.5 if torch.cuda.is_available() else 0},
    raise_on_failed_trial=False,
)

# Final result
if analysis.best_config:
    print("🎯 Best hyperparameters found:", analysis.best_config)
else:
    print("⚠️ No successful trials. But now your model is ready to return loss correctly.")


Using device: cuda


2025-07-11 05:28:38,649	WARNING services.py:2152 -- WARNING: The object store is using /tmp instead of /dev/shm because /dev/shm has only 67059712 bytes available. This will harm performance! You may be able to free up space by deleting files in /dev/shm. If you are inside a Docker container, you can increase /dev/shm size by passing '--shm-size=10.24gb' to 'docker run' (or add it to the run_options list in a Ray cluster config). Make sure to set this to more than 30% of available RAM.
2025-07-11 05:28:39,776	INFO worker.py:1917 -- Started a local Ray instance.
2025-07-11 05:28:40,383	INFO tune.py:616 -- [output] This uses the legacy output and progress reporter, as Jupyter notebooks are not supported by the new engine, yet. For more information, please see https://github.com/ray-project/ray/issues/36949


(train_with_tune pid=175878) /usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
(train_with_tune pid=175878)   warnings.warn(


(train_with_tune pid=175877) 🔧 Starting trial with config: {'batch_size': 2, 'lr': 1.2607329607942673e-05, 'epochs': 1}
(train_with_tune pid=175877) 📊 Train size: 293, Eval size: 74
(train_with_tune pid=175878) 🔧 Starting trial with config: {'batch_size': 1, 'lr': 9.263536712014789e-05, 'epochs': 1}


  0%|          | 0/147 [00:00<?, ?it/s]
(train_with_tune pid=175877) /usr/local/lib/python3.12/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
(train_with_tune pid=175877)   warnings.warn(
  3%|▎         | 5/147 [00:00<00:12, 11.39it/s]


...

(train_with_tune pid=175878) {'loss': 2.978, 'grad_norm': 0.0, 'learning_rate': 1.233030483851798e-05, 'epoch': 0.89} [repeated 38x across cluster]


100%|██████████| 293/293 [00:11<00:00, 25.83it/s]
2025-07-11 05:29:09,287	INFO tune.py:1009 -- Wrote the latest version of all result files and experiment state to '/root/ray_results/train_with_tune_2025-07-11_05-28-40' in 0.0040s.
2025-07-11 05:29:09,291	INFO tune.py:1041 -- Total run time: 28.91 seconds (28.89 seconds for the tuning loop).


🎯 Best hyperparameters found: {'batch_size': 1, 'lr': 9.263536712014789e-05, 'epochs': 1}


100%|██████████| 10/10 [00:00<00:00, 116.93it/s]
(train_with_tune pid=175878) /usr/local/lib/python3.12/dist-packages/ray/train/_internal/session.py:772: RayDeprecationWarning: `ray.train.report` should be switched to `ray.tune.report` when running in a function passed to Ray Tune. This will be an error in the future. See this issue for more context: https://github.com/ray-project/ray/issues/49454
(train_with_tune pid=175878)   _log_deprecation_warning(


(train_with_tune pid=175878) 📉 Eval results: {'eval_loss': 4.020482540130615, 'eval_runtime': 0.0979, 'eval_samples_per_second': 755.745, 'eval_steps_per_second': 102.128, 'epoch': 1.0}
